# 锚框

* 目标检测算法通常会在输入图像中采样大量的区域，然后判断这些区域中是否包含我们感兴趣的目标，并调整区域边界从而更准确地预测目标的*真实边界框*（ground-truth bounding box）
* 阅读：锚框.pdf

# 锚框

*锚框*（anchor box）

以每个像素为中心，生成多个缩放比和宽高比（aspect ratio）不同的边界框

## 生成多个锚框

* Let the *scale* be $s\in (0, 1]$ and the *aspect ratio* (ratio of width to height) is $r > 0$
* 锚框的宽度和高度分别是$ws\sqrt{r}$和$hs/\sqrt{r}$

## 生成多个锚框

* $s_1,\ldots, s_n$和$r_1,\ldots, r_m$
* 总共有$whnm$个锚框
* 尽管这些锚框可能会覆盖所有真实边界框，但计算复杂性很容易过高

## 生成多个锚框

* 在实践中，只考虑包含$s_1$或$r_1$的组合

  \begin{equation*}
  (s_1, r_1), (s_1, r_2), \ldots, (s_1, r_m), (s_2, r_1), (s_3, r_1), \ldots, (s_n, r_1)
  \end{equation*}
  
* 共生成$wh(n+m-1)$个锚框

## `multibox_prior`关键代码分析

1. 
    ```
    offset_h, offset_w = 0.5, 0.5
    steps_h = 1.0 / in_height  # 在y轴上缩放步长
    ```
1. 
    ```
    center_h = (torch.arange(in_height, 
        device=device) + offset_h) * steps_h
    ```

分析：细胞的高度$\tfrac{1}{h}$、第0个中心$y$坐标：$0.5\times \tfrac{1}{h}$、第$h-1$个中心$y$坐标：$0.5\times \tfrac{1}{h}+(h-1)\tfrac{1}{h}$

## `multibox_prior`关键代码分析

```
w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]),
               sizes[0] * torch.sqrt(ratio_tensor[1:])))\
               * in_height / in_width  # 处理矩形输入
```

`in_height / in_width` is there to handle rectangular inputs since ssd was originally developed for square images

## 交并比（IoU）

* 衡量锚框和真实边界框之间的相似性
* In fact, we can consider the pixel area of any bounding box as a set of pixels
* 阅读：锚框.pdf

## `box_iou`关键代码分析

```
inter_upperlefts = torch.max(boxes1[:, None, :2], 
                        boxes2[:, :2])
inter_lowerrights = torch.min(boxes1[:, None, 2:], 
                        boxes2[:, 2:])
    
inters = (inter_lowerrights - inter_upperlefts).clamp(min=0)
``` 

要点：广播、`.clamp(min=0)`

## `box_iou`关键代码分析

```
inter_areas = inters[:, :, 0] * inters[:, :, 1]
```

## 在训练数据中标注锚框

* 将每个锚框视为一个训练样本
* 需要每个锚框的*类别*（class）和*偏移量*（offset）标签
* *偏移量*（offset）：真实边界框相对于锚框的偏移量

## 将真实边界框分配给锚框

* 锚框是$A_1, A_2, \ldots, A_{n_a}$，真实边界框是$B_1, B_2, \ldots, B_{n_b}$，其中$n_a \geq n_b$
* $\mathbf{X} \in \mathbb{R}^{n_a \times n_b}$，$x_{ij}$是锚框$A_i$和真实边界框$B_j$的IoU

### 算法

1. 在矩阵$\mathbf{X}$中找到最大的元素，丢弃矩阵中${i_1}^\mathrm{th}$行和${j_1}^\mathrm{th}$列中的所有元素
1. 依次进行，直到为$n_b$个锚框各自分配了一个真实边界框
1. 遍历剩下的$n_a - n_b$个锚框，找到与其的IoU最大的真实边界框（大于预定义的阈值）

阅读：锚框.pdf

### `assign_anchor_to_bbox`关键代码分析

```
max_ious, indices = torch.max(jaccard, dim=1)
anc_i = torch.nonzero(max_ious >= 0.5).reshape(-1)
box_j = indices[max_ious >= 0.5]
anchors_bbox_map[anc_i] = box_j
```

`anchors_bbox_map`中默认为-1

-1：“背景”（background）

### `assign_anchor_to_bbox`关键代码分析

```
max_idx = torch.argmax(jaccard)
box_idx = (max_idx % num_gt_boxes).long()
anc_idx = (max_idx / num_gt_boxes).long()
anchors_bbox_map[anc_idx] = box_idx
```

要点：`torch.argmax`针对the *flattened* input

## 标记类别和偏移量

* 锚框$A$的偏移量将根据$B$和$A$中心坐标的相对位置以及这两个框的相对大小进行标记

\begin{equation*}
\biggl( \frac{ \frac{x_b - x_a}{w_a} - \mu_x }{\sigma_x},
\frac{ \frac{y_b - y_a}{h_a} - \mu_y }{\sigma_y},
\frac{ \log \frac{w_b}{w_a} - \mu_w }{\sigma_w},
\frac{ \log \frac{h_b}{h_a} - \mu_h }{\sigma_h}\biggr)
\end{equation*}

$\mu_x = \mu_y = \mu_w = \mu_h = 0, \sigma_x=\sigma_y=0.1$ ， $\sigma_w=\sigma_h=0.2$

### `offset_boxes`关键代码分析

```
offset_xy = 10 * (c_assigned_bb[:, :2] 
                    - c_anc[:, :2]) / c_anc[:, 2:]
offset_wh = 5 * torch.log(eps 
                    + c_assigned_bb[:, 2:] / c_anc[:, 2:])
```

### 标记类别

* “背景”（background）：一个锚框没有被分配真实边界框
* “负类”锚框

## 使用非极大值抑制预测边界框

* 预测边界框$B$的*置信度*（confidence）：$p$
* 排序列表$L$：所有预测的*非背景*边界框按置信度降序排序
* 阅读：锚框.pdf

### `nms`关键代码阅读

```
i = B[0]

iou = box_iou(boxes[i, :].reshape(-1, 4),
              boxes[B[1:], :].reshape(-1, 4)).reshape(-1)
inds = torch.nonzero(iou <= iou_threshold).reshape(-1)
B = B[inds + 1]
```

要点：计算`iou`时传入的为`B[1:]`，后续对`B`索引时应为`inds + 1`

## 小结

阅读：锚框.pdf